In [42]:
# from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
# import time
# import os
# import sys
# sys.path.append(os.path.dirname(os.getcwd()))
# sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

# import numpy as np
# import aeon
# import torch
# from torch import Tensor
# import torch.nn as nn
# import torch.nn.functional as F
# import pandas as pd
# from aeon.datasets.tsc_datasets import univariate_equal_length, multivariate_equal_length
# univariate_equal_length = sorted(list(univariate_equal_length))
# multivariate_equal_length = sorted(list(multivariate_equal_length))
# from aeon.datasets import load_classification
# from sklearn.linear_model import RidgeCV, RidgeClassifierCV
# from sklearn.metrics import accuracy_score
# from tqdm import tqdm

# from utils.utils import print_name, print_shape
# from preprocessing.stream_transforms import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
# from random_sig_fourier import SigTensorisedRandProj
# from signature import SigTransform, LogSigTransform
# from features.base import TimeseriesFeatureExtractor, TabularTimeseriesFeatures, RandomGuesser
# from randomized_sig import RandomizedSignature
# from rocket_wrappers import RocketWrapper, MiniRocketWrapper, MultiRocketWrapper

# np.set_printoptions(precision=3, threshold=5) # Print options

In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2Model

# Load pretrained model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2Model.from_pretrained(model_name)

# Move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [7]:
# Example input text
input_text = "zxzOnce upon a time, in a land far away,"

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors='pt')
print(inputs)

# Move input tensors to the GPU
inputs = {key: value.to(device) for key, value in inputs.items()}
print(inputs)

{'input_ids': tensor([[42592,    89,  7454,  2402,   257,   640,    11,   287,   257,  1956,
          1290,  1497,    11]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[42592,    89,  7454,  2402,   257,   640,    11,   287,   257,  1956,
          1290,  1497,    11]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [8]:
# Forward pass through the model
with torch.no_grad():
    outputs = model(**inputs)

# Extract the last hidden states
last_hidden_states = outputs.last_hidden_state

# last_hidden_states shape: (batch_size, sequence_length, hidden_size)
print(last_hidden_states.shape)

# Convert to CPU for further analysis if needed
last_hidden_states = last_hidden_states.cpu()

# Example: Print hidden states for each token
for i, token in enumerate(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])):
    print(f"Token: {token}, Hidden State: {last_hidden_states[0, i, :].numpy()}")

torch.Size([1, 13, 768])
Token: zx, Hidden State: [-2.10771978e-01  3.35755832e-02 -4.22556877e-01 -8.90451204e-03
  6.03716038e-02 -5.68474010e-02  2.49945164e+00  1.10583670e-01
 -2.23753035e-01 -3.03725861e-02  3.93025875e-01 -1.50372759e-01
 -1.00391962e-01 -1.18618915e-02 -1.17369860e-01 -1.64534301e-01
 -2.81927228e-01 -8.53329450e-02  2.00486854e-01 -2.26530522e-01
  2.43437439e-02 -8.19944069e-02 -1.72819063e-01 -2.71948371e-02
 -3.42614874e-02  2.34926790e-02 -2.53773540e-01 -9.47071686e-02
 -3.33473086e-03 -1.87961370e-01  3.23666353e-03  8.19651559e-02
 -6.72648177e-02 -3.36188078e-01 -1.53931990e-01 -9.72348917e-03
  2.03121567e+01  2.59849038e-02 -1.43527478e-01  2.83435136e-02
 -1.09205149e-01 -6.13129884e-02  4.96924669e-02 -2.11435601e-01
 -1.73964590e-01  3.49358283e-02 -1.40135452e-01 -7.61276037e-02
 -1.54812485e-01 -6.01132177e-02 -2.10834737e-03  1.84680924e-01
 -1.83456063e-01 -4.37023826e-02  9.21335816e-02  1.68337524e-01
  7.67951384e-02 -1.15188606e-01 -1.2057

In [10]:
######################################
#### my own input ####################
######################################

import torch
from torch.nn import Linear
from transformers import GPT2Model

# Load the largest GPT-2 model
model_name = 'gpt2'
model = GPT2Model.from_pretrained(model_name)

# Move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Example time series data (batch, time, dim)
N, T, D = 4, 50, 10  # Modify these dimensions as needed
time_series_data = torch.randn(N, T, D).float().to(device)

# Define a linear projection to match the GPT-2 hidden size
hidden_size = model.config.n_embd  # 1600 for gpt2-xl, 768 for gpt2
projection = Linear(D, hidden_size).to(device)

# Project the time series data to match GPT-2 input dimension
projected_data = projection(time_series_data)

# Forward pass through the model
with torch.no_grad():
    outputs = model(inputs_embeds=projected_data)

# Extract the last hidden states
last_hidden_states = outputs.last_hidden_state

# last_hidden_states shape: (N, T, hidden_size)
print(last_hidden_states.shape)

# Convert to CPU for further analysis if needed
last_hidden_states = last_hidden_states.cpu()

# Example: Print hidden states for each time step
for i in range(T):
    print(f"Time step: {i}, Hidden State: {last_hidden_states[0, i, :].numpy()}")

torch.Size([4, 50, 768])
Time step: 0, Hidden State: [-1.88379422e-01  2.88727582e-01 -3.17941517e-01 -3.91316898e-02
  3.61065269e-01 -6.33927286e-01  1.29547477e+00  3.83051664e-01
  1.90584511e-01 -1.64625987e-01  5.54312110e-01 -1.85160697e-01
 -2.12790385e-01 -1.89158618e-01 -5.01938999e-01  1.28912464e-01
 -2.74518192e-01 -1.04516514e-01  5.72531521e-01 -5.71382523e-01
  8.88535231e-02 -1.88606098e-01 -3.90094593e-02  1.74221396e-01
  9.01684165e-02  2.49613315e-01  1.30446434e-01  4.75126654e-02
 -2.77303550e-02 -7.19962120e-02 -2.59046435e-01 -2.65662432e-01
 -2.02809602e-01 -3.22278678e-01  3.83900434e-01 -4.88296188e-02
  2.69695606e+01  3.69195752e-02  2.11535007e-01  1.32312894e-01
  2.94269115e-01  3.35991859e-01 -9.95021909e-02 -3.01265478e-01
  2.49019727e-01  1.35515213e-01  2.44538337e-01 -3.63661617e-01
  2.76331604e-01 -2.68127322e-01  1.75763607e-01  4.87975270e-01
 -6.83548391e-01 -1.31914034e-01  2.02719215e-02 -1.08266883e-01
  4.22298193e-01 -2.17560843e-01 -3.2